#Using the basic structure from Analytics Vidhya: https://www.analyticsvidhya.com/blog/2019/09/step-by-step-deep-learning-tutorial-video-classification-python/

In [15]:
%matplotlib auto

Using matplotlib backend: MacOSX


playing around with cv2.videocapture, separating into frames. 

In [3]:
import cv2
import numpy as np

cap = cv2.VideoCapture('epilepsy/train/0.mp4')
frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

buf = np.empty((frameCount, frameHeight, frameWidth, 3), np.dtype('uint8'))

fc = 0
ret = True

while (fc < frameCount  and ret):
    ret, buf[fc] = cap.read()
    fc += 1

cap.release()





In [4]:
#cycle thru each frame
for i in range(len(buf)-1):
    cv2.imshow("frame " + str(i), buf[i])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [36]:
 cv2.destroyAllWindows()

beginning of the tutorial (https://www.analyticsvidhya.com/blog/2019/09/step-by-step-deep-learning-tutorial-video-classification-python/)

importing required libraries: 

In [5]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm

Using TensorFlow backend.


setting up the training dataframe

##setup training dataframe 
f = open("epilepsy/train/trainingList", "r")
temp = f.read()
trainingVideos = temp.split('\n')

train = pd.DataFrame()
train['video_name'] = trainingVideos
#removes unnecessary index column
train = train[:-1]
train.head()




create a tags column with the feature im testing for

In [7]:
#set tags: epilepsy-causing or no?
tag_column = []
for i in range(train.shape[0]):
    if train['video_name'][i][0] == 'n':
        tag_column.append(False)
    else:
        tag_column.append(True)
train['tag'] = tag_column
train

,video_name,tag
0,0.mp4,True
1,1.mp4,True
2,2.mp4,True
3,3.mp4,True
4,4.mp4,True
5,5.mp4,True
6,6.mp4,True
7,7.mp4,True
8,8.mp4,True
9,9.mp4,True


extracting each video's component frames and collecting them as a set of images

In [8]:
for i in tqdm(range(train.shape[0])):
    count = 0
    videoFile = train['video_name'][i]
    cap = cv2.VideoCapture('epilepsy/train/'+videoFile)   # capturing the video from the given path
    frameCount = cap.get(5) #frame count
#     frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    x=1
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (count%2 == 0):
            filename ='epilepsy/train_1/' + videoFile +"_frame%d.jpg" % (count/2)
            cv2.imwrite(filename, frame)
        count+=1
#         if (frameId % math.floor(frameCount) == 0):
#             # storing the frames in a new folder named train_1
#             filename ='epilepsy/train_1/' + videoFile +"_frame%d.jpg" % count;count+=1
#             cv2.imwrite(filename, frame)
    cap.release()

100%|██████████| 24/24 [00:31<00:00,  1.32s/it]


creating a dataframe with each image and its class, converting to a csv

In [10]:
images = glob("epilepsy/train_1/*.jpg")
train_image = []
train_class = []
for i in tqdm(range(len(images))):
    # creating the image name
    train_image.append(images[i].split('/')[2])
    # creating the class of image
    if(images[i].split('/')[2][0] == 'n'):
        train_class.append("no")
    else:
        train_class.append("yes")
    
# storing the images and their class in a dataframe
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class

# converting the dataframe into csv file 
train_data.to_csv('epilepsy/train_new.csv',header=True, index=False)

100%|██████████| 1556/1556 [00:00<00:00, 243048.45it/s]


import libraries required to read and interpret frames

In [11]:
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

create df from images csv, rename column

In [20]:
train = pd.read_csv('epilepsy/train_new.csv')
train.rename(columns = {'class' : 'trigger'}, inplace = True)
train.head()

,image,trigger
0,n1.mp4_frame18.jpg,no
1,n9.mp4_frame410.jpg,no
2,n9.mp4_frame376.jpg,no
3,n9.mp4_frame362.jpg,no
4,n9.mp4_frame404.jpg,no


create a numpy array of the images, 

In [22]:
train_image = []

for i in tqdm(range(train.shape[0])):
    #load image
    img = image.load_img('epilepsy/train_1/'+train['image'][i], target_size=(224,224,3))
    #convert to array
    img = image.img_to_array(img)
    #normalize pixel values
    img = img/255
    #add to list of images
    train_image.append(img)
    
X = np.array(train_image)
X.shape

100%|██████████| 1556/1556 [00:16<00:00, 95.09it/s] 


(1556, 224, 224, 3)

npArray.shape gives the number of elements in each dimension. (1556, 224, 224, 3) means the outermost array has 1556 elements (one for each image), and each image contains 224 row indices, which have 224 column indices, which have 3 values for r g and b respectively. 

example: X[0][0][0] should give us a 3-length array of rgb values, which correspond to the first pixel of the first image. 

In [23]:
X[0][0][0]

array([0.23921569, 0.13725491, 0.13333334], dtype=float32)

# NOTES

Tools used: 
tqdm: progress bars for For Loops 
    i.e. for i in tqdm(range(0,10))
glob: retrieve files/pathnames that match a pattern
    images = glob("epilepsy/train_1/*.jpg")


Pandas notes: 
    pd.read_csv --> converts csv to a dataframe using columns as listed in csv
    pd.describe() --> shows number of uniques in each column, range of each column, etc. 
    pd.head() --> top 5 entries in dataframe

OpenCV notes: 
    